## 1.0 Importing Packages

#### 1.1 Import standard packages

In [5]:
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.use('agg')
import matplotlib.pyplot as plt
import time
import emcee
import corner

#### 1.2 Install + Import GALARIO

In [3]:
conda install -c conda-forge galario

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - galario


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fftw-3.3.10                |nompi_hc118613_108         1.9 MB  conda-forge
    galario-1.2.2              |py38h7191ed2_1003         315 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  fftw               conda-forge/linux-64::fftw-3.3.10-nompi_hc118613_108 
  galario            conda-forge/linux-64::galario-1.2.2-py38h7191ed2_1003 



fftw-3.3.10          | 1.9 MB    |                                       |   0% 
fftw-3.3.10          | 1.9 MB    | ##################################### | 100% 
galario-1.2

In [4]:
import galario.double as g_double
from galario import arcsec, deg

## 2.0 Define Functions

In [2]:
def lnpriorfn(p, par_ranges):

    """ Uniform prior probability function """

    for i in range(len(p)):
        if p[i] < par_ranges[i][0] or p[i] > par_ranges[i][1]:
            return -np.inf

    jacob = -p[0]

    return jacob

In [6]:
def lnpostfn(p, p_ranges, rmin, dr, nr, nxy, dxy, u, v, re, im, w):

    """ Log of posterior probability function """

    ### apply prior
    lnprior = lnpriorfn(p, p_ranges)
    if not np.isfinite(lnprior):
        return -np.inf

    ### unpack the parameters
    if args.r_in:
        f0, sigma, r_in, inc, pa, dra, ddec = p
        r_in *= arcsec
    else:
        f0, sigma, inc, pa, dra, ddec = p

    ### convert to correct units
    f0 = 10.**f0
    sigma *= arcsec
    inc *= deg
    pa *= deg
    dra *= arcsec
    ddec *= arcsec

    ### get gaussian profile
    f, r = GaussianProfile(f0, sigma, rmin, dr, nr)

    ### add inner cavity?
    if args.r_in:
        f[r < r_in] *= 0

    ### calculate chi-squared
    chi2 = g_double.chi2Profile(f, rmin, dr, nxy, dxy, u, v, re, im, w,
                                inc=inc, PA=pa, dRA=dra, dDec=ddec)

    ### return likelihood
    return -0.5 * chi2 + lnprior

In [7]:
def GaussianProfile(f0, sigma, rmin, dr, nr):

    """ Gaussian brightness profile. """

    ### calculate radial grid
    r = np.linspace(rmin, rmin + dr * nr, nr, endpoint=False)

    ### calculate gaussian profile
    f = f0 * np.exp(-0.5 * (r / sigma)**2)

    ### return gaussian profile
    return f, r

## 3.0 Main Code

In [11]:
### setup args
args_nsteps = 3000
args_nthreads = 5
args_f0, args_sigma = 10.0, 1.0
args_incl, args_pa = 30.0, 40.0
args_dra, args_ddec = 0.000001, 0.00001
args_r_in = 20.0
p0 = [args_f0, args_sigma, args_r_in, args_incl, args_pa, args_dra, args_ddec]

### print to screen
print(f"\nInputs:\n\tIncl = {args_incl} deg\n\tPA = {args_pa} deg\n\tR_in = {args_r_in} AU\n")


Inputs:
	Incl = 30.0 deg
	PA = 40.0 deg
	R_in = 20.0 AU



In [12]:
### setup mcmc
ndim = len(p0)
nwalkers = ndim * 20 
nsteps = int(args_nsteps / 5)
tsteps = args_nsteps
nthreads = args_nthreads
print(f"\nEmcee setup:\n\tSteps = {tsteps}\n\tWalkers = {nwalkers}\n\tThreads = {nthreads}\n")


Emcee setup:
	Steps = 3000
	Walkers = 140
	Threads = 5



In [ ]:
### read in UV table visibilities
print("\nReading in UV table: " + args.uvtable)
U, V, Re, Im, W = np.loadtxt(args.uvtable, unpack=True)
U, V = np.ascontiguousarray(U), np.ascontiguousarray(V)
U /= args.wavelength
V /= args.wavelength